In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
inputPath = "D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

['False' 'True']
2


In [9]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



for i in filesArray:
    print(i)

1150
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_191958.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192022.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192026.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192043.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192126.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192132.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192140.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192153.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192209.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\D

In [10]:
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    
    data = data/2 #Normalise

    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1, 0]
    else:
        label = [0, 1]
    labels.append(label)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)


In [5]:
import datetime

# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model = Sequential()
model.add(L.Input((1000, 1), name='input_accel', dtype="float32"))
'''
# Rebuild Top
model.add(L.Conv1D(32,3, input_shape = (1000,1)))
model.add(L.Conv1D(8,8, input_shape = (1000,1)))
model.add(L.Dense(128, activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dense(32, activation="relu"))
model.add(L.Dropout(0.25))
model.add(L.GlobalMaxPooling1D())
model.add(L.Dense(2, activation="softmax"))
'''
model.add(L.Conv1D(32,150, activation=L.LeakyReLU(alpha=0.7)))
model.add(L.Dropout(0.2))

model.add(L.Conv1D(64,64, activation=L.LeakyReLU(alpha=0.5)))
model.add(L.Dropout(0.2))

model.add(L.Conv1D(128,3, activation=L.LeakyReLU(alpha=0.3)))
model.add(L.Dropout(0.2))

model.add(L.Conv1D(256,2, activation=L.LeakyReLU(alpha=0.2)))

model.add(L.Conv1D(512,2, activation=L.LeakyReLU(alpha=0.1)))

model.add(L.GlobalMaxPooling1D())
model.add(L.Dropout(0.2))
model.add(L.Flatten())

model.add(L.Dense(128, activation=L.LeakyReLU(alpha=0.1)))
model.add(L.Dense(2, activation="softmax"))

model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

# model.build((235, 4001, 1))
print(model.summary())


df_array = np.array(df)
labels_array = np.array(labels)

model.fit(df_array, labels_array, shuffle=1, epochs=40, validation_split=0.05, batch_size = 1, callbacks=[tensorboard_callback])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 851, 32)           4832      
                                                                 
 dropout (Dropout)           (None, 851, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 788, 64)           131136    
                                                                 
 dropout_1 (Dropout)         (None, 788, 64)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 786, 128)          24704     
                                                                 
 dropout_2 (Dropout)         (None, 786, 128)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 785, 256)          6

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(df_array, labels_array, shuffle=1, epochs=40, validation_split=0.05, batch_size = 1, callbacks=[tensorboard_callback])

In [44]:
model.evaluate(df_array, labels_array, batch_size=1)

1150/1150 [==============================] - 5s 4ms/step - loss: 0.0362 - categorical_accuracy: 0.9852


[0.036208223551511765, 0.9852173924446106]

In [40]:
import tensorflow_model_optimization as tfmot

# Load your Keras model
model = tf.keras.models.load_model('model.h5')

# Define the pruning parameters
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.1,
    final_sparsity=0.5,
    begin_step=0,
    end_step=1000
)

# Define the pruning callback
pruning_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir='/tmp')
]

# Create a pruned model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)

# Train the pruned model
pruned_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
pruned_model.fit(df_array, labels_array, shuffle=1, epochs=10, validation_split=0.25, callbacks=pruning_callbacks)

# Remove the pruning wrappers from the model
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Save the pruned model
final_model.save('pruned_model.h5')

Epoch 1/10
27/27 [==============================] - 12s 323ms/step - loss: 0.1396 - accuracy: 0.9478 - val_loss: 0.1470 - val_accuracy: 0.9375
Epoch 2/10
27/27 [==============================] - 9s 317ms/step - loss: 0.1538 - accuracy: 0.9420 - val_loss: 0.1183 - val_accuracy: 0.9514
Epoch 3/10
27/27 [==============================] - 8s 301ms/step - loss: 0.1092 - accuracy: 0.9582 - val_loss: 0.1680 - val_accuracy: 0.9097
Epoch 4/10
27/27 [==============================] - 8s 316ms/step - loss: 0.1149 - accuracy: 0.9501 - val_loss: 0.1266 - val_accuracy: 0.9479
Epoch 5/10
27/27 [==============================] - 8s 314ms/step - loss: 0.0645 - accuracy: 0.9780 - val_loss: 0.0922 - val_accuracy: 0.9514
Epoch 6/10
27/27 [==============================] - 8s 314ms/step - loss: 0.0958 - accuracy: 0.9675 - val_loss: 0.1753 - val_accuracy: 0.9167
Epoch 7/10
27/27 [==============================] - 8s 305ms/step - loss: 0.0718 - accuracy: 0.9664 - val_loss: 0.0698 - val_accuracy: 0.9653
Epoch

In [66]:
final_model = tf.keras.models.load_model('pruned_model.h5')
tf.keras.models.save_model(final_model, 'pruned_model', save_format='tf')

INFO:tensorflow:Assets written to: pruned_model\assets


INFO:tensorflow:Assets written to: pruned_model\assets


In [45]:
final_model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
model.evaluate(df_array, labels_array, batch_size=1)
final_model.evaluate(df_array, labels_array, batch_size=1)

1150/1150 [==============================] - 5s 4ms/step - loss: 0.0362 - categorical_accuracy: 0.9852


[0.036208223551511765, 0.9852173924446106]

In [18]:
model.save("model1.h5")

In [20]:
tf.keras.models.save_model(model, 'model1', save_format='tf')

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets


In [14]:
model_copy = model

In [67]:
converter = tf.lite.TFLiteConverter.from_saved_model("pruned_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


with open('my_model_quantized.tflite', 'wb') as f:
  f.write(tflite_model)

In [68]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [69]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

In [70]:
print(input_details[0]['shape'])
print(df_array.shape)
#old_df = df_array

[   1 1000    1]
(1150, 1000, 1)


In [71]:

df_array = np.expand_dims(old_df, axis=-1) # add extra dimension to the end

In [95]:
correct = 0
total = 0
from scipy.stats import pearsonr



for i in range(len(df_array)):
    interpreter.set_tensor(input_details[0]['index'], df_array[i:i+1])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    origional = model.predict(df_array[i:i+1])
        
    print(f'Label: {labels_array[i]}    tf model prediction: {origional}    tflite model prediction: {output}')
    prediction = np.argmax(output)
    mape ,  _ = pearsonr(origional[0], output[0])
    print(mape)
    if(mape > 0.95):
            correct += 1
    
    total += 1
    print(f'{correct} of {total} predictions were accurate to within 5%')

accuracy = correct / total
print('Accuracy:', accuracy)

1/1 [==============================] - 0s 15ms/step
Label: [1 0]    tf model prediction: [[1.0000000e+00 2.8591039e-08]]    tflite model prediction: [[9.9998987e-01 1.0124718e-05]]
1.0
1 of 1 predictions were accurate to within 5%
1/1 [==============================] - 0s 15ms/step
Label: [0 1]    tf model prediction: [[0.00509968 0.9949004 ]]    tflite model prediction: [[0.00279667 0.99720335]]
1.0
2 of 2 predictions were accurate to within 5%
Accuracy: 1.0
